# Week 3 Part 3

In [1]:
import pandas as pd

In [8]:
# Read prepared data from CSV
yyz_latlon = pd.read_csv('yyz_neighbour_latlon.csv')

# Keep only those rows have 'Toronto' in their Borough column
yyz = yyz_latlon[yyz_latlon['Borough'].str.contains("Toronto")]
yyz.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
